In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Libraries

In [0]:
import pandas as pd
import math 

Import Dataset

In [0]:
data_temp = pd.read_csv('/content/drive/My Drive/AOBD_Practical_2020/ID3_data.csv', header=None)
data_temp.head()

,0,1,2,3,4
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes


Give proper name to the columns

In [0]:
data = data_temp.rename(columns={0:'age',1:'income',2:'student',3:'credit_rating',4:'buys_computer'})
data.head()

,age,income,student,credit_rating,buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes


![alt text](https://drive.google.com/uc?id=1RfX_IHHifmZ6ssdYjSa65u0fZ_u1dcDk)

In [0]:
def calculate_sufficient_statistic(data):


    #calculate number of different values of each attribute
    class_cnt = []
    total_attribute = len(data.columns)
    attr_name = data.columns
    for i in range(total_attribute):
        class_cnt.insert(i,data[attr_name[i]].unique())


    #sufficient statistic column name like age:youth
    suffi_statastic_colname =[]
    for i in range(total_attribute-1):
        cnt = len(class_cnt[i])

        for j in range(cnt):
            suffi_statastic_colname.append(attr_name[i]+':'+class_cnt[i][j])


    #pute all class of attribute in one list like [[age:youth,age:middle_aged,age:senior],...]
    attr_name_class = []
    for i in attr_name:
        attr_name_class.append([col for col in suffi_statastic_colname if i in col])

    attr_name_class=attr_name_class[:-1]
    #print(attr_name_class)

    #convert that into series
    attr_name_class = pd.Series(attr_name_class,index=attr_name[:-1])
    #print(attr_name_class)
        

    #calculate sufficient statistic
    suffi_state =[]
    output_class = class_cnt[-1]
    output_class_name = attr_name[-1]
    for i in range(len(suffi_statastic_colname)):

        temp_list=[]
        str_split = suffi_statastic_colname[i].split(':')
        #print(str_split)
        #data[data[str_split[0]]==str_split[1]]

        for j in output_class:

            temp_list.append(len(data[(data[str_split[0]]==str_split[1]) & (data[output_class_name]==j)]))

        suffi_state.append(temp_list)


    #put all sufficient statistic into dataframe
    temp_list=[]
    for i in output_class:
        temp_list.append(len(data[data[attr_name[-1]]==i]))
    suffi_state.append(temp_list)


    temp=[]
    for i in range(len(output_class)):
        temp.append([x[i] for x in suffi_state])

    #print(temp)
    df_state = pd.DataFrame(temp,columns=suffi_statastic_colname+['output_class'],index = output_class)
    df_state.head()


    return attr_name,attr_name_class,suffi_statastic_colname,df_state



![alt text](https://drive.google.com/uc?id=1pnwuDpprgcr012tiQc2I2tP6XVgfJgh4)

In [0]:
def calculate_entropy(attr,df_suffi_state,attr_class):
    
    info = 0
    D = sum(df_suffi_state['output_class'])
    #print('D = ',D)
    for i in range(len(attr_class)):
        Dj = sum(df_suffi_state[attr_class[i]])
        #print('Dj = ',Dj)
        temp_sum=0
        for j in range(len(df_suffi_state)):
            d = df_suffi_state[attr_class[i]][j]
            #print('d = ',d)
            if d==0:
                temp_sum+=0
            else:
                temp_sum += -(d/Dj)*math.log(d/Dj,2)
        
        info += (Dj/D)*temp_sum

    #print(info)
    return info


In [0]:
#calculate_entropy('age',df_state,attr_name_class['age'])

In [0]:
def best_spliting_attribute(df_state,attr_name_class,attr_name):
    
    #attr_name = df_state.columns

    info =[]

    for i in attr_name:
        if i == 'buys_computer' or i=='output_class':
            continue
        #print(i)
        #print(attr_name_class[i])
        info.append(calculate_entropy(i,df_state,attr_name_class[i]))

    min_info_index = info.index(min(info))

    #print(attr_name[min_info_index])

    return attr_name[min_info_index]



In [0]:
#best_spliting_attribute(df_state,attr_name_class,attr_name)
best_attr_list = []
dic = {}
tree={}

In [0]:
def ID3(data,tree=None):
    
    
    #print('data: ',data)

    [attr_name,attr_name_class,suffi_statastic_colname,df_state] = calculate_sufficient_statistic(data)

    #print('attr name : ',attr_name)
    #print('attr name class : ',attr_name_class)
    #print('suffi_statastic_colname : ',suffi_statastic_colname)
    #print('df_state : ',df_state)
    

    best_attr = best_spliting_attribute(df_state,attr_name_class,attr_name)
    best_attr_list.append(best_attr)

    #print('best_attr:',best_attr)

    best_attr_class = [col for col in suffi_statastic_colname if best_attr in col]

    '''
    if attr_key == '':
        temp_dic = {}
        
        for i in best_attr_class:
            temp_split = i.split(':')
            temp_dic[temp_split[1]] = {}

        dic[best_attr] = temp_dic
    else:
        attr_split = attr_key.split(':')

        print(attr_split)
        d_temp = dic
        for key in attr_split:
            print(key)
            d_temp = d_temp[key]
            print(d_temp)
        temp_dic = {}
        
        for i in best_attr_class:
            temp_split = i.split(':')
            temp_dic[temp_split[1]] = {}

        d_temp = temp_dic
    
    if tree is None:                    
        tree={}
        tree[best_attr] = {}
    '''   
    if tree is None:                    
        tree={}
        tree[best_attr] = {}

    #print('best_attr_class : ',best_attr_class)

    for attr_class in best_attr_class:
        #print(attr_class)
        temp_split = attr_class.split(':')
        temp_data = data[data[temp_split[0]]==temp_split[1]]
        temp_data = temp_data.drop([temp_split[0]], axis = 1)

        #print('temp_data : ',temp_data)
        #print('attr_class : ',attr_class)
        if len(temp_data['buys_computer'].unique())>1:
            #print('temp_data : ',temp_data)
            #return ID3(temp_data)
            tree[best_attr][temp_split[1]]=ID3(temp_data)
        else:
            #print(temp_data['buys_computer'].unique()[0])
            #print(tree)
            tree[best_attr][temp_split[1]] = temp_data['buys_computer'].unique()[0]
    
    return tree


In [86]:
ID3(data)

{'age': {'middle_aged': 'yes',
  'senior': {'credit_rating': {'excellent': 'no', 'fair': 'yes'}},
  'youth': {'student': {'no': 'no', 'yes': 'yes'}}}}

In [0]:
d = {'age': {'middle_aged': {}, 'senior': {'hhh':1}, 'youth': {}}}
d['age']['senior']

{'hhh': 1}

In [0]:
attr_key = 'age:middle_aged'
attr_split = attr_key.split(':')

print(attr_split)
d_temp = d
for key in attr_split:
    print(key)
    d_temp = d_temp[key]
    print(d_temp)
d_temp = 3
print(d)
print(d_temp)

['age', 'middle_aged']
age
{'middle_aged': {}, 'senior': {'hhh': 1}, 'youth': {}}
middle_aged
{}
{'age': {'middle_aged': {}, 'senior': {'hhh': 1}, 'youth': {}}}
3


In [0]:
d['age']['middle_aged']=9

In [0]:
d

{'age': {'middle_aged': 9, 'senior': {'hhh': 1}, 'youth': {}}}